In [ ]:
!pip install pandas-gbq
pip install yfinance
import pandas as pd
import yfinance as yf
import requests
import json 
import csv
import datetime
import os
import glob
import io
from google.cloud import bigquery
from google.oauth2 import service_account


Manually load list of tickers

In [ ]:
data = '''AMZN
INTC
LRCX
META
BABA
GIS
KMI
KO
MCD
MDT
MMM
TEL
USB
XYL
URNM
ABNB
AMAT
ASML
KLAC
KLIC
MRAM
SWKS
UROY
VMFXX
BWXT
DOCS
SPOT
SILXY
CLLNY
SRUUF
YLLXF
IGN
VGT
VUG
SOXX
VONG
JEPI
SCHD
VB
VIG
VTI
FXAIX
SPAXX
VXUS
TWCGX
SPY
EMCLOUD
COMP
FROG
MCHP
IRDM
NVEC
MXL
OPRA
BELFB
EXTR
IDCC
ON
HLIT
PLAB
VEEV
TTD
SHOP
CDLX
ZI
DV
S
ZS
CRWD
ADSK
PANW
MSI
ANET
EQIX
EGIO
DOCN
NET
RKLB
ASTS
COIN
NCNO
FISV
ENPH
TOELY
QRVO
BESI
VSH
AEHR
TXN
SLAB
CRUS
NXPI
AVGO
MPWR
LSCC
SNPS
CDNS
APH
CGNX
PLPC
SYM
IOT
TMO
DHR
TT
IRM
SEIC
NOW
BDC
DIOD
JBL
CARG
CSGS
ADBE
CRM
NOW
SHOP
WDAY
PYPL
SNOW
CRWD
TEAM
DDOG
SQ
MDB
VEEV
ZS
HUBS
NET
ZM
OKTA
TWLO
PAYC
PATH
DBX
PCTY
DOCU
TOST
ESTC
PCOR
APPF
GTLB
MNDY
QLYS
BILL
SMAR
CFLT
FRSH
WIX
ZI
WK
S
FIVN
BRZE
TENB
ASAN
SQSP
CXM
CWAN
BOX
BL
NCNO
AI
SPT
FROG
RNG
DOCN
FSLY
PD
QTWO
AVDX
AVPT
AMPL
ZUO
EVBG
DH
YEXT
OLO
BIGC
VMEO
WEAV
EGHT
DOMO
NOVT
IPGP
COHR
GFS
MU
WOLF
TXN
INTC
NVDA
ADI
QCOM
NXPI
TER
ALGM
MRVL
SWKS
ASML
AMD
MPWR
ON
LSCC
QRVO
AVGO
MCHP
AMAT
LRCX
ENTG
KLAC
TSM
SYNA
MKL'''

tickers = data.split('\n')
print(tickers)

Full Year Data

# set the API endpoint URL with a placeholder for the ticker
url_template = "https://fmpcloud.io/api/v3/ratios/{}?datatype=csv&apikey=90b7ef69ff8550aabc5480eccc68a854"

# define a list of tickers
tickers = tickers

# define an empty dictionary to store the dataframes for each ticker
ticker_dataframes = {}

# loop over the tickers and make an API request for each one
for ticker in tickers:
    # replace the placeholder with the current ticker
    url = url_template.format(ticker)

    # make an HTTP GET request to the API endpoint and retrieve the data in CSV format
    response = requests.get(url)
    data = response.text

    # create a dataframe from the data
    df = pd.read_csv(io.StringIO(data))
    
    # Remove the 'label' column
    #df.drop('calendarYear', axis=1, inplace=True)

    # store the dataframe in the ticker_dataframes dictionary with the ticker as the key
    ticker_dataframes[ticker] = df

# concatenate all dataframes into one
combined_df = pd.concat(ticker_dataframes.values(), ignore_index=True)

# save the combined dataframe as a CSV file
combined_df.to_csv("ratiosFY.csv", index=False)

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

# Read the CSV file into a DataFrame
ratios = pd.read_csv('ratiosFY.csv')


# Handle Non-finite Values:
# ratios['averageInventory'] = ratios['averageInventory'].fillna(0).astype(int)

# Drop the 'calendarYear' column if it exists
if 'CALENDARYEAR' in ratios.columns:
    ratios.drop(columns=['CALENDARYEAR'], inplace=True)

# Explicitly set the 'DATE' column as datetime64[ns]
ratios = pd.DataFrame({
    'SYMBOL': ratios['symbol'].astype(str),
    'DATE': pd.to_datetime(ratios['date'], errors='coerce'),
    'PERIOD': ratios['period'].astype(str),
    'CURRENTRATIO': ratios['currentRatio'].astype(float),
    'QUICKRATIO': ratios['quickRatio'].astype(float),
    'CASHRATIO': ratios['cashRatio'].astype(float),
    'DAYSOFSALESOUTSTANDING': ratios['daysOfSalesOutstanding'].astype(float),
    'DAYSOFINVENTORYOUTSTANDING': ratios['daysOfInventoryOutstanding'].astype(float),
    'OPERATINGCYCLE': ratios['operatingCycle'].astype(float),
    'DAYSOFPAYABLESOUTSTANDING': ratios['daysOfPayablesOutstanding'].astype(float),
    'CASHCONVERSIONCYCLE': ratios['cashConversionCycle'].astype(float),
    'GROSSPROFITMARGIN': ratios['grossProfitMargin'].astype(float),
    'OPERATINGPROFITMARGIN': ratios['operatingProfitMargin'].astype(float),
    'PRETAXPROFITMARGIN': ratios['pretaxProfitMargin'].astype(float),
    'NETPROFITMARGIN': ratios['netProfitMargin'].astype(float),
    'EFFECTIVETAXRATE': ratios['effectiveTaxRate'].astype(float),
    'RETURNONASSETS': ratios['returnOnAssets'].astype(float),
    'RETURNONEQUITY': ratios['returnOnEquity'].astype(float),
    'RETURNONCAPITALEMPLOYED': ratios['returnOnCapitalEmployed'].astype(float),
    'NETINCOMEPEREBT': ratios['netIncomePerEBT'].astype(float),
    'EBTPEREBIT': ratios['ebtPerEbit'].astype(float),
    'EBITPERREVENUE': ratios['ebitPerRevenue'].astype(float),
    'DEBTRATIO': ratios['debtRatio'].astype(float),
    'DEBTEQUITYRATIO': ratios['debtEquityRatio'].astype(float),
    'LONGTERMDEBTTOCAPITALIZATION': ratios['longTermDebtToCapitalization'].astype(float),
    'TOTALDEBTTOCAPITALIZATION': ratios['totalDebtToCapitalization'].astype(float),
    'INTERESTCOVERAGE': ratios['interestCoverage'].astype(float),
    'CASHFLOWTODEBTRATIO': ratios['cashFlowToDebtRatio'].astype(float),
    'COMPANYEQUITYMULTIPLIER': ratios['companyEquityMultiplier'].astype(float),
    'RECEIVABLESTURNOVER': ratios['receivablesTurnover'].astype(float),
    'PAYABLESTURNOVER': ratios['payablesTurnover'].astype(float),
    'INVENTORYTURNOVER': ratios['inventoryTurnover'].astype(float),
    'FIXEDASSETTURNOVER': ratios['fixedAssetTurnover'].astype(float),
    'ASSETTURNOVER': ratios['assetTurnover'].astype(float),
    'OPERATINGCASHFLOWPERSHARE': ratios['operatingCashFlowPerShare'].astype(float),
    'FREECASHFLOWPERSHARE': ratios['freeCashFlowPerShare'].astype(float),
    'CASHPERSHARE': ratios['cashPerShare'].astype(float),
    'PAYOUTRATIO': ratios['payoutRatio'].astype(float),
    'OPERATINGCASHFLOWSALESRATIO': ratios['operatingCashFlowSalesRatio'].astype(float),
    'FREECASHFLOWOPERATINGCASHFLOWRATIO': ratios['freeCashFlowOperatingCashFlowRatio'].astype(float),
    'CASHFLOWCOVERAGERATIOS': ratios['cashFlowCoverageRatios'].astype(float),
    'SHORTTERMCOVERAGERATIOS': ratios['shortTermCoverageRatios'].astype(float),
    'CAPITALEXPENDITURECOVERAGERATIO': ratios['capitalExpenditureCoverageRatio'].astype(float),
    'DIVIDENDPAIDANDCAPEXCOVERAGERATIO': ratios['dividendPaidAndCapexCoverageRatio'].astype(float),
    'DIVIDENDPAYOUTRATIO': ratios['dividendPayoutRatio'].astype(float),
    'PRICEBOOKVALUERATIO': ratios['priceBookValueRatio'].astype(float),
    'PRICETOBOOKRATIO': ratios['priceToBookRatio'].astype(float),
    'PRICETOSALESRATIO': ratios['priceToSalesRatio'].astype(float),
    'PRICEEARNINGSRATIO': ratios['priceEarningsRatio'].astype(float),
    'PRICETOFREECASHFLOWSRATIO': ratios['priceToFreeCashFlowsRatio'].astype(float),
    'PRICETOOPERATINGCASHFLOWSRATIO': ratios['priceToOperatingCashFlowsRatio'].astype(float),
    'PRICECASHFLOWRATIO': ratios['priceCashFlowRatio'].astype(float),
    'PRICEEARNINGSTOGROWTHRATIO': ratios['priceEarningsToGrowthRatio'].astype(float),
    'PRICESALESRATIO': ratios['priceSalesRatio'].astype(float),
    'DIVIDENDYIELD': ratios['dividendYield'].astype(float),
    'ENTERPRISEVALUEMULTIPLE': ratios['enterpriseValueMultiple'].astype(float),
    'PRICEFAIRVALUE': ratios['priceFairValue'].astype(float),
})

ratios.columns = ratios.columns.str.upper()


project_id = 'stock-market-data-391622'
dataset_id = 'market_information'
table_name = 'ratios'


# Create a BigQuery client with service account credentials
credentials = service_account.Credentials.from_service_account_file('stock-market-data-391622-03ee9e592be9.json')
client = bigquery.Client(credentials=credentials, project=project_id)

# Specify the destination table in BigQuery
destination_table = 'stock-market-data-391622.market_information.ratios'


# Write the DataFrame to BigQuery without specifying the schema
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # Options: "WRITE_TRUNCATE", "WRITE_APPEND", "WRITE_EMPTY", "WRITE_TRUNCATE"
)


client.load_table_from_dataframe(
    ratios, destination_table, job_config=job_config
).result()

In [ ]:
job_id = '9dd0f12d-69cb-42d0-b705-6a439f906d88'  # Replace with new job ID
job = client.get_job(job_id)
print(job.state)

Quarterly Data

In [ ]:
# set the API endpoint URL with a placeholder for the ticker
url_template = "https://fmpcloud.io/api/v3/ratios/{}?datatype=csv&period=quarter&apikey=a726d5acabd86910ef2749315f5eaa86"

# define a list of tickers
tickers = tickers

# define an empty dictionary to store the dataframes for each ticker
ticker_dataframes = {}

# loop over the tickers and make an API request for each one
for ticker in tickers:
    # replace the placeholder with the current ticker
    url = url_template.format(ticker)

    # make an HTTP GET request to the API endpoint and retrieve the data in CSV format
    response = requests.get(url)
    data = response.text

    # create a dataframe from the data
    df = pd.read_csv(io.StringIO(data))
    
    # Remove the 'label' column
    #df.drop('calendarYear', axis=1, inplace=True)

    # store the dataframe in the ticker_dataframes dictionary with the ticker as the key
    ticker_dataframes[ticker] = df

# concatenate all dataframes into one
combined_df = pd.concat(ticker_dataframes.values(), ignore_index=True)

# save the combined dataframe as a CSV file
combined_df.to_csv("ratios_quarterly.csv", index=False)

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

# Read the CSV file into a DataFrame
ratios = pd.read_csv('ratios_quarterly.csv')


# Handle Non-finite Values:
# ratios['averageInventory'] = ratios['averageInventory'].fillna(0).astype(int)

# Drop the 'calendarYear' column if it exists
if 'CALENDARYEAR' in ratios.columns:
    ratios.drop(columns=['CALENDARYEAR'], inplace=True)

# Explicitly set the 'DATE' column as datetime64[ns]
ratios = pd.DataFrame({
    'SYMBOL': ratios['symbol'].astype(str),
    'DATE': pd.to_datetime(ratios['date'], errors='coerce'),
    'PERIOD': ratios['period'].astype(str),
    'CURRENTRATIO': ratios['currentRatio'].astype(float),
    'QUICKRATIO': ratios['quickRatio'].astype(float),
    'CASHRATIO': ratios['cashRatio'].astype(float),
    'DAYSOFSALESOUTSTANDING': ratios['daysOfSalesOutstanding'].astype(float),
    'DAYSOFINVENTORYOUTSTANDING': ratios['daysOfInventoryOutstanding'].astype(float),
    'OPERATINGCYCLE': ratios['operatingCycle'].astype(float),
    'DAYSOFPAYABLESOUTSTANDING': ratios['daysOfPayablesOutstanding'].astype(float),
    'CASHCONVERSIONCYCLE': ratios['cashConversionCycle'].astype(float),
    'GROSSPROFITMARGIN': ratios['grossProfitMargin'].astype(float),
    'OPERATINGPROFITMARGIN': ratios['operatingProfitMargin'].astype(float),
    'PRETAXPROFITMARGIN': ratios['pretaxProfitMargin'].astype(float),
    'NETPROFITMARGIN': ratios['netProfitMargin'].astype(float),
    'EFFECTIVETAXRATE': ratios['effectiveTaxRate'].astype(float),
    'RETURNONASSETS': ratios['returnOnAssets'].astype(float),
    'RETURNONEQUITY': ratios['returnOnEquity'].astype(float),
    'RETURNONCAPITALEMPLOYED': ratios['returnOnCapitalEmployed'].astype(float),
    'NETINCOMEPEREBT': ratios['netIncomePerEBT'].astype(float),
    'EBTPEREBIT': ratios['ebtPerEbit'].astype(float),
    'EBITPERREVENUE': ratios['ebitPerRevenue'].astype(float),
    'DEBTRATIO': ratios['debtRatio'].astype(float),
    'DEBTEQUITYRATIO': ratios['debtEquityRatio'].astype(float),
    'LONGTERMDEBTTOCAPITALIZATION': ratios['longTermDebtToCapitalization'].astype(float),
    'TOTALDEBTTOCAPITALIZATION': ratios['totalDebtToCapitalization'].astype(float),
    'INTERESTCOVERAGE': ratios['interestCoverage'].astype(float),
    'CASHFLOWTODEBTRATIO': ratios['cashFlowToDebtRatio'].astype(float),
    'COMPANYEQUITYMULTIPLIER': ratios['companyEquityMultiplier'].astype(float),
    'RECEIVABLESTURNOVER': ratios['receivablesTurnover'].astype(float),
    'PAYABLESTURNOVER': ratios['payablesTurnover'].astype(float),
    'INVENTORYTURNOVER': ratios['inventoryTurnover'].astype(float),
    'FIXEDASSETTURNOVER': ratios['fixedAssetTurnover'].astype(float),
    'ASSETTURNOVER': ratios['assetTurnover'].astype(float),
    'OPERATINGCASHFLOWPERSHARE': ratios['operatingCashFlowPerShare'].astype(float),
    'FREECASHFLOWPERSHARE': ratios['freeCashFlowPerShare'].astype(float),
    'CASHPERSHARE': ratios['cashPerShare'].astype(float),
    'PAYOUTRATIO': ratios['payoutRatio'].astype(float),
    'OPERATINGCASHFLOWSALESRATIO': ratios['operatingCashFlowSalesRatio'].astype(float),
    'FREECASHFLOWOPERATINGCASHFLOWRATIO': ratios['freeCashFlowOperatingCashFlowRatio'].astype(float),
    'CASHFLOWCOVERAGERATIOS': ratios['cashFlowCoverageRatios'].astype(float),
    'SHORTTERMCOVERAGERATIOS': ratios['shortTermCoverageRatios'].astype(float),
    'CAPITALEXPENDITURECOVERAGERATIO': ratios['capitalExpenditureCoverageRatio'].astype(float),
    'DIVIDENDPAIDANDCAPEXCOVERAGERATIO': ratios['dividendPaidAndCapexCoverageRatio'].astype(float),
    'DIVIDENDPAYOUTRATIO': ratios['dividendPayoutRatio'].astype(float),
    'PRICEBOOKVALUERATIO': ratios['priceBookValueRatio'].astype(float),
    'PRICETOBOOKRATIO': ratios['priceToBookRatio'].astype(float),
    'PRICETOSALESRATIO': ratios['priceToSalesRatio'].astype(float),
    'PRICEEARNINGSRATIO': ratios['priceEarningsRatio'].astype(float),
    'PRICETOFREECASHFLOWSRATIO': ratios['priceToFreeCashFlowsRatio'].astype(float),
    'PRICETOOPERATINGCASHFLOWSRATIO': ratios['priceToOperatingCashFlowsRatio'].astype(float),
    'PRICECASHFLOWRATIO': ratios['priceCashFlowRatio'].astype(float),
    'PRICEEARNINGSTOGROWTHRATIO': ratios['priceEarningsToGrowthRatio'].astype(float),
    'PRICESALESRATIO': ratios['priceSalesRatio'].astype(float),
    'DIVIDENDYIELD': ratios['dividendYield'].astype(float),
    'ENTERPRISEVALUEMULTIPLE': ratios['enterpriseValueMultiple'].astype(float),
    'PRICEFAIRVALUE': ratios['priceFairValue'].astype(float),
})

ratios.columns = ratios.columns.str.upper()


project_id = 'stock-market-data-391622'
dataset_id = 'market_information'
table_name = 'ratios'


# Create a BigQuery client with service account credentials
credentials = service_account.Credentials.from_service_account_file('stock-market-data-391622-03ee9e592be9.json')
client = bigquery.Client(credentials=credentials, project=project_id)

# Specify the destination table in BigQuery
destination_table = 'stock-market-data-391622.market_information.ratios'


# Write the DataFrame to BigQuery without specifying the schema
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # Options: "WRITE_TRUNCATE", "WRITE_APPEND", "WRITE_EMPTY", "WRITE_TRUNCATE"
)


client.load_table_from_dataframe(
    ratios, destination_table, job_config=job_config
).result()

In [ ]:
job_id = 'c2e70840-7ebd-497f-8d75-2879cc91ebd0'  # Replace with new job ID
job = client.get_job(job_id)
print(job.state)